# Finite Difference Method for Numerical Differentiation and Differential Equations

## 1. Introduction

The **Finite Difference Method (FDM)** is a fundamental numerical technique for approximating derivatives and solving differential equations. It forms the backbone of computational physics, engineering simulations, and scientific computing.

### 1.1 Theoretical Foundation

The finite difference method is based on Taylor series expansion. For a sufficiently smooth function $f(x)$, we can write:

$$f(x + h) = f(x) + h f'(x) + \frac{h^2}{2!} f''(x) + \frac{h^3}{3!} f'''(x) + \mathcal{O}(h^4)$$

$$f(x - h) = f(x) - h f'(x) + \frac{h^2}{2!} f''(x) - \frac{h^3}{3!} f'''(x) + \mathcal{O}(h^4)$$

### 1.2 Finite Difference Approximations

#### Forward Difference (First-Order Accurate)
$$f'(x) \approx \frac{f(x + h) - f(x)}{h} + \mathcal{O}(h)$$

#### Backward Difference (First-Order Accurate)
$$f'(x) \approx \frac{f(x) - f(x - h)}{h} + \mathcal{O}(h)$$

#### Central Difference (Second-Order Accurate)
$$f'(x) \approx \frac{f(x + h) - f(x - h)}{2h} + \mathcal{O}(h^2)$$

#### Second Derivative (Central Difference)
$$f''(x) \approx \frac{f(x + h) - 2f(x) + f(x - h)}{h^2} + \mathcal{O}(h^2)$$

### 1.3 Application to Differential Equations

Consider the one-dimensional Poisson equation with Dirichlet boundary conditions:

$$\frac{d^2 u}{dx^2} = f(x), \quad x \in [a, b]$$

with $u(a) = u_a$ and $u(b) = u_b$.

Discretizing the domain into $N+1$ points with spacing $h = (b-a)/N$, we obtain:

$$\frac{u_{i+1} - 2u_i + u_{i-1}}{h^2} = f_i, \quad i = 1, 2, \ldots, N-1$$

This yields a tridiagonal linear system $\mathbf{A}\mathbf{u} = \mathbf{b}$.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.linalg import solve

# Set up plotting style
plt.rcParams['figure.figsize'] = [12, 10]
plt.rcParams['font.size'] = 11
plt.rcParams['axes.grid'] = True

## 2. Finite Difference Schemes for Numerical Differentiation

Let's implement and compare the three main finite difference schemes for computing derivatives.

In [ ]:
def forward_difference(f, x, h):
    """Forward difference approximation of f'(x)."""
    return (f(x + h) - f(x)) / h

def backward_difference(f, x, h):
    """Backward difference approximation of f'(x)."""
    return (f(x) - f(x - h)) / h

def central_difference(f, x, h):
    """Central difference approximation of f'(x)."""
    return (f(x + h) - f(x - h)) / (2 * h)

def second_derivative_central(f, x, h):
    """Central difference approximation of f''(x)."""
    return (f(x + h) - 2 * f(x) + f(x - h)) / (h ** 2)

### 2.1 Convergence Analysis

We'll test these schemes on $f(x) = \sin(x)$, where the exact derivative is $f'(x) = \cos(x)$.

In [ ]:
# Test function and its derivatives
f = lambda x: np.sin(x)
f_prime_exact = lambda x: np.cos(x)
f_double_prime_exact = lambda x: -np.sin(x)

# Point of evaluation
x0 = 1.0

# Range of step sizes
h_values = np.logspace(-1, -10, 50)

# Compute errors for each scheme
errors_forward = []
errors_backward = []
errors_central = []
errors_second = []

exact_first = f_prime_exact(x0)
exact_second = f_double_prime_exact(x0)

for h in h_values:
    errors_forward.append(abs(forward_difference(f, x0, h) - exact_first))
    errors_backward.append(abs(backward_difference(f, x0, h) - exact_first))
    errors_central.append(abs(central_difference(f, x0, h) - exact_first))
    errors_second.append(abs(second_derivative_central(f, x0, h) - exact_second))

errors_forward = np.array(errors_forward)
errors_backward = np.array(errors_backward)
errors_central = np.array(errors_central)
errors_second = np.array(errors_second)

## 3. Solving the Poisson Equation

We'll solve the boundary value problem:

$$\frac{d^2 u}{dx^2} = -\pi^2 \sin(\pi x), \quad x \in [0, 1]$$

with $u(0) = 0$ and $u(1) = 0$.

The exact solution is $u(x) = \sin(\pi x)$.

In [ ]:
def solve_poisson_fdm(N, a=0, b=1, ua=0, ub=0, source=None):
    """
    Solve the 1D Poisson equation using finite differences.
    
    Parameters:
    -----------
    N : int
        Number of interior grid points
    a, b : float
        Domain boundaries
    ua, ub : float
        Dirichlet boundary conditions
    source : callable
        Source function f(x)
    
    Returns:
    --------
    x : ndarray
        Grid points including boundaries
    u : ndarray
        Solution values at grid points
    """
    # Grid spacing
    h = (b - a) / (N + 1)
    
    # Interior grid points
    x_interior = np.linspace(a + h, b - h, N)
    
    # Construct the tridiagonal matrix A
    # A[i,i] = -2, A[i,i-1] = 1, A[i,i+1] = 1
    diagonal = -2 * np.ones(N)
    off_diagonal = np.ones(N - 1)
    
    A = np.diag(diagonal) + np.diag(off_diagonal, 1) + np.diag(off_diagonal, -1)
    A = A / (h ** 2)
    
    # Construct the right-hand side vector
    rhs = source(x_interior)
    
    # Apply boundary conditions
    rhs[0] -= ua / (h ** 2)
    rhs[-1] -= ub / (h ** 2)
    
    # Solve the linear system
    u_interior = solve(A, rhs)
    
    # Include boundary points
    x = np.concatenate([[a], x_interior, [b]])
    u = np.concatenate([[ua], u_interior, [ub]])
    
    return x, u

In [ ]:
# Define the problem
source_func = lambda x: -np.pi**2 * np.sin(np.pi * x)
exact_solution = lambda x: np.sin(np.pi * x)

# Solve with different grid resolutions
N_values = [5, 10, 20, 50, 100]
solutions = {}
errors_L2 = []
errors_Linf = []

for N in N_values:
    x, u = solve_poisson_fdm(N, source=source_func)
    solutions[N] = (x, u)
    
    # Compute errors
    u_exact = exact_solution(x)
    error = np.abs(u - u_exact)
    
    # L2 norm error
    h = 1.0 / (N + 1)
    L2_error = np.sqrt(h * np.sum(error**2))
    errors_L2.append(L2_error)
    
    # L-infinity norm error
    Linf_error = np.max(error)
    errors_Linf.append(Linf_error)
    
    print(f"N = {N:3d}: L2 error = {L2_error:.2e}, L∞ error = {Linf_error:.2e}")

## 4. Visualization and Results

In [ ]:
# Create comprehensive visualization
fig, axes = plt.subplots(2, 2, figsize=(14, 12))

# Plot 1: Convergence of finite difference schemes
ax1 = axes[0, 0]
ax1.loglog(h_values, errors_forward, 'b-', label='Forward Difference', linewidth=2)
ax1.loglog(h_values, errors_backward, 'r--', label='Backward Difference', linewidth=2)
ax1.loglog(h_values, errors_central, 'g-.', label='Central Difference', linewidth=2)
ax1.loglog(h_values, errors_second, 'm:', label='Second Derivative (Central)', linewidth=2)

# Reference lines for convergence rates
h_ref = h_values[10:30]
ax1.loglog(h_ref, 0.5 * h_ref, 'k--', alpha=0.5, label=r'$\mathcal{O}(h)$')
ax1.loglog(h_ref, 0.5 * h_ref**2, 'k:', alpha=0.5, label=r'$\mathcal{O}(h^2)$')

ax1.set_xlabel('Step size $h$', fontsize=12)
ax1.set_ylabel('Absolute Error', fontsize=12)
ax1.set_title('Convergence of Finite Difference Schemes', fontsize=14)
ax1.legend(loc='upper left', fontsize=9)
ax1.set_xlim([1e-10, 1e-1])
ax1.set_ylim([1e-16, 1e0])

# Plot 2: Poisson equation solutions
ax2 = axes[0, 1]
x_fine = np.linspace(0, 1, 200)
ax2.plot(x_fine, exact_solution(x_fine), 'k-', label='Exact', linewidth=2)

colors = plt.cm.viridis(np.linspace(0.2, 0.8, len(N_values)))
for i, N in enumerate(N_values):
    x, u = solutions[N]
    ax2.plot(x, u, 'o-', color=colors[i], label=f'N = {N}', 
             markersize=8-i, alpha=0.7)

ax2.set_xlabel('$x$', fontsize=12)
ax2.set_ylabel('$u(x)$', fontsize=12)
ax2.set_title('Poisson Equation: $u\'\'(x) = -\pi^2 \sin(\pi x)$', fontsize=14)
ax2.legend(loc='upper right', fontsize=9)

# Plot 3: Error decay with grid refinement
ax3 = axes[1, 0]
h_poisson = [1.0 / (N + 1) for N in N_values]

ax3.loglog(h_poisson, errors_L2, 'bo-', label='$L_2$ Error', linewidth=2, markersize=8)
ax3.loglog(h_poisson, errors_Linf, 'rs-', label='$L_\infty$ Error', linewidth=2, markersize=8)

# Reference line for O(h^2) convergence
h_ref2 = np.array(h_poisson)
ax3.loglog(h_ref2, 0.5 * h_ref2**2, 'k--', alpha=0.5, label=r'$\mathcal{O}(h^2)$')

ax3.set_xlabel('Grid spacing $h$', fontsize=12)
ax3.set_ylabel('Error', fontsize=12)
ax3.set_title('Grid Convergence for Poisson Equation', fontsize=14)
ax3.legend(loc='upper left', fontsize=10)

# Plot 4: Point-wise error distribution
ax4 = axes[1, 1]

for i, N in enumerate([10, 50, 100]):
    x, u = solutions[N]
    u_exact = exact_solution(x)
    error = np.abs(u - u_exact)
    ax4.semilogy(x, error + 1e-16, 'o-', label=f'N = {N}', 
                 markersize=6, alpha=0.7)

ax4.set_xlabel('$x$', fontsize=12)
ax4.set_ylabel('Point-wise Error $|u - u_{exact}|$', fontsize=12)
ax4.set_title('Spatial Distribution of Error', fontsize=14)
ax4.legend(loc='upper right', fontsize=10)

plt.tight_layout()
plt.savefig('plot.png', dpi=150, bbox_inches='tight')
plt.show()

print("\nPlot saved to 'plot.png'")

## 5. Summary and Conclusions

### Key Findings

1. **Truncation Error vs. Round-off Error**: The convergence plots reveal two distinct regimes:
   - For large $h$: truncation error dominates, decreasing as $\mathcal{O}(h)$ or $\mathcal{O}(h^2)$
   - For small $h$: round-off error dominates, causing error to increase

2. **Order of Accuracy**:
   - Forward/Backward differences: first-order accurate $\mathcal{O}(h)$
   - Central difference: second-order accurate $\mathcal{O}(h^2)$

3. **Poisson Equation Solution**: The finite difference method achieves second-order convergence, as expected from the central difference approximation of the second derivative.

### Practical Considerations

- **Optimal step size**: Balance between truncation and round-off errors
- **Matrix conditioning**: Tridiagonal systems are well-conditioned and efficiently solvable
- **Boundary conditions**: Proper incorporation is crucial for accuracy

### Extensions

The finite difference method extends naturally to:
- Higher-order schemes (fourth-order, compact schemes)
- Time-dependent problems (heat equation, wave equation)
- Multi-dimensional domains
- Non-uniform grids and adaptive refinement